In [17]:
from pathlib import Path
import yaml

import pandas as pd

# Add oof predictions to the dataset
Add out of fold predictions of models to train\test.

## Constants

In [ ]:
DATA_PATH = Path("../data/")
OOF_PATH = DATA_PATH / "models"
CONFIG_FILE = Path("../configs/config.yaml")

models = ["lamau_81425_full_dataset", "lgb_8122_full_dataset",
          "cb_8114_full_dataset", "xgb_81325_full_dataset",
         "lama_81298_full_dataset", "lamann_autoint_8053_full_dataset", "lamann_fttransformer_8050_full_dataset"]

## Add out of fold predictions to train/test

In [6]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc_2.parquet")
df_test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")

In [13]:
for model in models:
    oof_pred = pd.read_csv(OOF_PATH / model / "oof.csv")
    test_pred = pd.read_csv(OOF_PATH / model / f"{model}.csv")
    
    df_train[model] = oof_pred[model]
    df_test[model] = test_pred["target"]

In [ ]:
df_train.to_parquet(DATA_PATH / "train_preproc_oof.parquet", index=False)
df_test.to_parquet(DATA_PATH / "test_preproc_oof.parquet", index=False)

## Add names of oof columns to the config

In [18]:
with CONFIG_FILE.open("r") as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)

In [19]:
cfg["stack_features"] = models

In [20]:
with CONFIG_FILE.open("w") as f:
    yaml.dump(cfg, f)